# LAB 03: Query Parameters and Advanced Queries in Microsoft Graph PowerShell

This lab will help you learn how to use query parameters and advanced queries in the Microsoft Graph PowerShell SDK. You'll practice using filter expressions, sorting, pagination, and advanced query techniques to efficiently retrieve data from Microsoft Graph.

## Prerequisites

Before beginning this lab, ensure you have the following:

1. The Microsoft Graph PowerShell SDK installed
2. Appropriate permissions to query Microsoft Graph

Run the cell below to check your installation and connect to Microsoft Graph:

In [ ]:
# Check if already connected
$context = Get-MgContext
if (!$context) {
    # Connect to Microsoft Graph with required scopes
    Connect-MgGraph -Scopes "User.Read.All", "Group.Read.All", "Application.Read.All", "Directory.Read.All"
}

# Display connection information
Get-MgContext | Format-List AppName, TenantId, Scopes

## Understanding Query Parameters

Microsoft Graph API supports various query parameters that allow you to customize your requests:

- **$select**: Choose which properties to include in the results
- **$filter**: Filter results based on specific criteria
- **$orderby**: Sort results by one or more properties
- **$top**: Limit the number of items returned
- **$skip**: Skip a certain number of items
- **$count**: Include a count of items
- **$expand**: Include related resources
- **$search**: Search across specific properties

In PowerShell, these parameters are available as named parameters on the corresponding cmdlets.

## Lab Tasks

In this lab, you will complete the following tasks:

1. Order service principals in ascending order by 'DisplayName' and return the top 10 results
2. Find users without a job title
3. Find users with the Office 365 E5 licenses
4. Get all users without assigned licenses
5. Find security groups

For each task, we'll explore the approach, explain the solution, and provide the PowerShell commands.

## Task 1: Order Service Principals by DisplayName

In this task, we'll retrieve service principals ordered by their display name in ascending order and limit the results to the top 10.

### Understanding the Approach

To complete this task, we need to:
1. Use the `Get-MgServicePrincipal` cmdlet
2. Apply ordering with the `-OrderBy` parameter
3. Limit results with the `-Top` parameter
4. Use `-ConsistencyLevel eventual` and `-CountVariable` for advanced query support

### Step-by-step Solution

In [ ]:
# Step 1: Basic approach - This will NOT work correctly because OrderBy requires advanced query support
Write-Host "Attempting to order service principals without advanced query support:" -ForegroundColor Cyan
try {
    Get-MgServicePrincipal -OrderBy DisplayName -Top 10
} catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}

In [ ]:
# Step 2: Correct approach using advanced query parameters
Write-Host "Ordering service principals with advanced query support:" -ForegroundColor Green

# Order service principals in ascending order by DisplayName and return the top 10
$servicePrincipals = Get-MgServicePrincipal -OrderBy DisplayName -Top 10 -ConsistencyLevel eventual -CountVariable count

# Display results in a formatted table
$servicePrincipals | Select-Object DisplayName, Id, AppId | Format-Table -AutoSize

# Show total count for context
Write-Host "Returned $($servicePrincipals.Count) out of a total of $count service principals" -ForegroundColor Yellow

### Explanation

In the solution above:

1. We used `-OrderBy DisplayName` to sort the results in ascending order by display name
2. `-Top 10` limits the results to only 10 items
3. `-ConsistencyLevel eventual` enables advanced query capabilities
4. `-CountVariable count` stores the total count of matching items

The `-ConsistencyLevel eventual` parameter is required when using certain query options like `-OrderBy` with directory objects. This tells the API that we're willing to accept eventual consistency for more advanced query capabilities.

## Task 2: Find Users Without a Job Title

In this task, we'll find all users who don't have a job title specified.

### Understanding the Approach

To find users without a job title, we need to:
1. Use the `Get-MgUser` cmdlet
2. Apply a filter for users where `jobTitle eq null`
3. Use advanced query parameters since filtering on null values requires advanced query support

### Step-by-step Solution

In [ ]:
# Step 1: Attempt without advanced query support - This will NOT work
Write-Host "Attempting to find users without a job title without advanced query support:" -ForegroundColor Cyan
try {
    Get-MgUser -Filter "jobTitle eq null" -Top 3
} catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}

In [ ]:
# Step 2: Correct approach using advanced query parameters
Write-Host "Finding users without a job title with advanced query support:" -ForegroundColor Green

# Find users without a job title
$usersWithoutJobTitle = Get-MgUser -Filter "jobTitle eq null" -ConsistencyLevel eventual -CountVariable count -Top 10

# Display results
$usersWithoutJobTitle | Select-Object DisplayName, UserPrincipalName | Format-Table

# Show total count
Write-Host "Found $count users without a job title (showing first 10)" -ForegroundColor Yellow

### Explanation

In the solution above:

1. We used `-Filter "jobTitle eq null"` to find users without a job title
2. The `-ConsistencyLevel eventual` parameter was required because filtering on null values is an advanced query feature
3. `-CountVariable count` stores the total count of matching users

Note: Filtering on null values is only supported when using advanced query parameters with `-ConsistencyLevel eventual`.

## Task 3: Find Users with Office 365 E5 Licenses

In this task, we'll find users who have been assigned the Office 365 E5 license, which has the SKU ID `c42b9cae-ea4f-4ab7-9717-81576235ccac`.

### Understanding the Approach

To find users with a specific license, we need to:
1. Use the `Get-MgUser` cmdlet
2. Apply a filter that checks the `assignedLicenses` **collection**
3. Use the `any` operator to check if any item in the collection matches our criteria

### Step-by-step Solution

In [ ]:
# Step 1: Define the Office 365 E5 SKU ID
$e5SkuId = "c42b9cae-ea4f-4ab7-9717-81576235ccac"

# Step 2: Find users with the Office 365 E5 license using the any operator
Write-Host "Finding users with Office 365 E5 licenses:" -ForegroundColor Green
try {
    # Method 1: Hardcoded SKU ID in the filter
    $usersWithE5_method1 = Get-MgUser -Filter "assignedLicenses/any(x:x/skuId eq c42b9cae-ea4f-4ab7-9717-81576235ccac)" -All
    
    # Method 2: Using a variable in the filter
    $usersWithE5_method2 = Get-MgUser -Filter "assignedLicenses/any(x:x/skuId eq $e5SkuId)" -All
    
    # Display results
    $usersWithE5_method2 | Select-Object DisplayName, UserPrincipalName | Format-Table
    
    # Show total count
    Write-Host "Found $($usersWithE5_method2.Count) users with Office 365 E5 licenses" -ForegroundColor Yellow
} catch {
    Write-Host "Error: $($_.Exception.Message)" -ForegroundColor Red
}

### Explanation

In the solution above:

1. We defined the Office 365 E5 SKU ID as a variable for clarity
2. We used `-Filter "assignedLicenses/any(x:x/skuId eq $e5SkuId)"` to find users with that license
3. The filter uses the `any` operator to check each item in the assignedLicenses collection
4. We showed two methods: hardcoding the GUID and using a variable

The `/any()` operator is used for filtering on collections. The syntax is:
```
collectionProperty/any(variable:variable/subProperty eq 'value')
```
collectionProperty: The collection-valued property you are filtering on.

variable: An iterator representing each item in the collection during evaluation.

subProperty: A property of the collection item to evaluate.

'value': The value to compare against

## Task 4: Get All Users Without Assigned Licenses

In this task, we'll find users who don't have any licenses assigned.

### Understanding the Approach

To find users without licenses, we need to:
1. Use the `Get-MgUser` cmdlet
2. Check if the assignedLicenses collection is empty using `$count eq 0`
3. Use advanced query parameters for this type of filter

### Step-by-step Solution

In [ ]:
# Step 1: Incorrect approach that will fail
Write-Host "Attempting an incorrect approach:" -ForegroundColor Cyan
try {
    Get-MgUser -Filter "assignedLicenses/any(s:s/skuId eq null)" -Top 3 -ConsistencyLevel eventual -CountVariable countIncorrect -ea 0
} catch {
    Write-Error "Error: $($_.Exception.Message)"
}

In [ ]:
# Step 2: Correct approach using $count eq 0
Write-Host "Finding users without licenses (correct approach):" -ForegroundColor Green

# Need to escape $ in PowerShell string
$usersWithoutLicenses = Get-MgUser -Filter "assignedLicenses/`$count eq 0" -ConsistencyLevel eventual -CountVariable countVar -Top 10

# Display results
$usersWithoutLicenses | Select-Object DisplayName, UserPrincipalName | Format-Table

# Show total count
Write-Host "Found $countVar users without licenses (showing first 10)" -ForegroundColor Yellow

### Explanation

In the solution above:

1. We first showed an incorrect approach that tries to use `eq null` inside an `any` clause, which isn't supported
2. The correct approach uses ``-Filter "assignedLicenses/`$count eq 0"`` to find users with an empty assignedLicenses collection
3. Note the backtick (\`) used to escape the dollar sign in the PowerShell string
4. We used `-ConsistencyLevel eventual` because this type of filter requires advanced query support

The `/$count eq 0` syntax is a special filter that checks if a collection property is empty. This is the proper way to find users with no licenses assigned.

## Task 5: Find Security Groups

In this task, we'll find all security-enabled groups in the tenant.

### Understanding the Approach

To find security groups, we need to:
1. Use the `Get-MgGroup` cmdlet
2. Apply a filter for groups where `securityEnabled eq true`

### Step-by-step Solution

In [ ]:
# Find security groups
Write-Host "Finding security groups:" -ForegroundColor Green

# Simple filter for security-enabled groups
$securityGroups = Get-MgGroup -Filter "securityEnabled eq true" -All

# Display results
$securityGroups | Select-Object DisplayName, Description, Id | Format-Table

# Show total count
Write-Host "Found $($securityGroups.Count) security groups" -ForegroundColor Yellow

### Explanation

In the solution above:

1. We used `-Filter "securityEnabled eq true"` to find security-enabled groups
2. The `-All` parameter ensures we get all matching groups, not just the default page
3. The `securityEnabled` is a boolean property, so we filter with `eq true`

This filter is simpler than the others and doesn't require advanced query support because it's a straightforward comparison of a boolean property.

## Additional Task: Combining Filters for More Complex Queries

Let's explore how to combine filters for more complex scenarios.

### Example: Find security groups that are also mail-enabled

In [ ]:
# Find security groups that are also mail-enabled
Write-Host "Finding security groups that are also mail-enabled:" -ForegroundColor Green

# Combine filters with logical operators
$securityMailGroups = Get-MgGroup -Filter "securityEnabled eq true and mailEnabled eq true" -All

# Display results
$securityMailGroups | Select-Object DisplayName, Description, Id | Format-Table

# Show total count
Write-Host "Found $($securityMailGroups.Count) security groups that are also mail-enabled" -ForegroundColor Yellow

### Example: Find Guest Users Without Job Titles

In [ ]:
# Find guest users without job titles
Write-Host "Finding guest users without job titles:" -ForegroundColor Green

# Combine multiple conditions
$guestUsersWithoutJobTitle = Get-MgUser -Filter "userType eq 'Guest' and jobTitle eq null" -ConsistencyLevel eventual -CountVariable countGuests

# Display results
$guestUsersWithoutJobTitle | Select-Object DisplayName, UserPrincipalName | Format-Table

# Show total count
Write-Host "Found $countGuests guest users without job titles" -ForegroundColor Yellow

## Summary

In this lab, you've learned how to:

1. Use `-OrderBy` parameter to sort results
2. Use `-Filter` parameter with various operators and conditions
3. Work with null values in filters
4. Use the `/any()` operator to filter on collection properties
5. Use the `/$count` operator to check if a collection is empty
6. Enable advanced query capabilities with `-ConsistencyLevel eventual`
7. Combine multiple conditions in filters

These techniques will help you efficiently query and retrieve data from Microsoft Graph using the PowerShell SDK.

## Clean Up

When you're done with the lab, disconnect from Microsoft Graph:

In [ ]:
Disconnect-MgGraph